In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import langchain
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [3]:
from pathlib import Path

docs_dir = Path('/home/jinzy/work/automation/privateGPT/source_documents')

loader = DirectoryLoader(docs_dir, glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()
print(len(documents))

158


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(len(texts))

743


In [6]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                                      model_kwargs={"device": "cuda"})
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory)

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [8]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=instructor_embeddings)
retriever = vectordb.as_retriever()

In [9]:
docs = retriever.get_relevant_documents("What is Flash attention?")
print(len(docs))
docs[0]

4


Document(page_content='access.\nWe propose FlashAttention , a new attention algorithm that computes exact attention with far fewer\nmemory accesses. Our main goal is to avoid reading and writing the attention matrix to and from HBM.\nThis requires (i) computing the softmax reduction without access to the whole input (ii) not storing the large\nintermediate attention matrix for the backward pass. We apply two well-established techniques to address\nthese challenges. (i) We restructure the attention computation to split the input into blocks and make several\npasses over input blocks, thus incrementally performing the softmax reduction (also known as tiling). (ii) We\nstore the softmax normalization factor from the forward pass to quickly recompute attention on-chip in the\nbackward pass, which is faster than the standard approach of reading the intermediate attention matrix from\nHBM. We implement FlashAttention in CUDA to achieve ﬁne-grained control over memory access and', metadata={'

In [10]:
docs = retriever.get_relevant_documents("请解释一下什么是flash attention?")
print(len(docs))
docs[0]

4


Document(page_content='access.\nWe propose FlashAttention , a new attention algorithm that computes exact attention with far fewer\nmemory accesses. Our main goal is to avoid reading and writing the attention matrix to and from HBM.\nThis requires (i) computing the softmax reduction without access to the whole input (ii) not storing the large\nintermediate attention matrix for the backward pass. We apply two well-established techniques to address\nthese challenges. (i) We restructure the attention computation to split the input into blocks and make several\npasses over input blocks, thus incrementally performing the softmax reduction (also known as tiling). (ii) We\nstore the softmax normalization factor from the forward pass to quickly recompute attention on-chip in the\nbackward pass, which is faster than the standard approach of reading the intermediate attention matrix from\nHBM. We implement FlashAttention in CUDA to achieve ﬁne-grained control over memory access and', metadata={'

In [ ]:
# from llama_index.embeddings.base import BaseEmbedding
# from langchain.embeddings import HuggingFaceInstructEmbeddings

# class InstructEmbedding(BaseEmbedding):
#     def __init__(self, handle: Optional[str] = None) -> None:
#         """Init params."""
#         try:
#             self._instructor = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
#                                                              model_kwargs={"device": "cuda"})
#         except ImportError:
#             raise ImportError(
#                 "Please install InstructorEmbeddings"
#             )

#         super().__init__()

#     def _get_query_embedding(self, query: str) -> List[float]:
#         """Get query embedding."""
#         return self._get_embedding(query)

#     def _get_text_embedding(self, text: str) -> List[float]:
#         """Get text embedding."""
#         return self._get_embedding(text)

#     def _get_embedding(self, text: str) -> List[float]:
#         vectors = self._instruct([text]).numpy().tolist()
#         return vectors[0]

In [12]:
import tiktoken
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter


node_parser = SimpleNodeParser(text_splitter=text_splitter)
service_context = ServiceContext.from_defaults(
    embed_model=LangchainEmbedding(instructor_embeddings), node_parser=node_parser
)

documents = SimpleDirectoryReader(docs_dir, recursive=True, required_exts=['.txt', '.pdf']).load_data()
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [13]:
query_engine = index.as_query_engine()
query_engine.query('请解释一下什么是flash attention?')

Response(response='\nFlashAttention is an IO-aware exact attention algorithm that uses tiling to reduce the number of memory reads/writes between GPU high bandwidth memory (HBM) and GPU on-chip SRAM. It is designed to reduce the amount of HBM access required for sequence length, compared to standard attention. FlashAttention also serves as a useful primitive for realizing the potential of approximate attention algorithms by overcoming their issues with memory access overhead.', source_nodes=[NodeWithScore(node=Node(text='in sequence length—than standard attention, thanks to the massively reduced amount of HBM access.\nWe analyze the IO complexity [ 1] ofFlashAttention , proving that it requires 𝑂¹𝑁2𝑑2𝑀\x001ºHBM\naccesses where 𝑑is the head dimension and 𝑀is the size of SRAM, as compared to Ω¹𝑁𝑑¸𝑁2ºof standard\nattention. For typical values of 𝑑and𝑀,FlashAttention requires many times fewer HBM accesses\ncompared to standard attention (up to 9 \x02fewer, as shown in Fig. 2). Moreover, we

In [14]:
query_engine.query('请用中文解释一下什么是flash attention?')

Response(response='\nFlash Attention是一种IO感知的精确注意力算法，它使用平铺来减少GPU高带宽存储器（HBM）和GPU片上SRAM之间的读/写次数。它比标准注意力更有效地利用HBM访问，并且可以提供更长的序列长度。', source_nodes=[NodeWithScore(node=Node(text='in sequence length—than standard attention, thanks to the massively reduced amount of HBM access.\nWe analyze the IO complexity [ 1] ofFlashAttention , proving that it requires 𝑂¹𝑁2𝑑2𝑀\x001ºHBM\naccesses where 𝑑is the head dimension and 𝑀is the size of SRAM, as compared to Ω¹𝑁𝑑¸𝑁2ºof standard\nattention. For typical values of 𝑑and𝑀,FlashAttention requires many times fewer HBM accesses\ncompared to standard attention (up to 9 \x02fewer, as shown in Fig. 2). Moreover, we provide a lower bound,\nshowing that no exact attention algorithm can asymptotically improve on the number of HBM accesses over\nall SRAM sizes.\nWe also show that FlashAttention can serve as a useful primitive for realizing the potential of\napproximate attention algorithms by overcoming their issues with memory access overhead. As a proof of\nconcep

In [15]:
query_engine.query('请用中文解释一下什么是flash attention?假设我是5岁小孩，尽可能直观、详细')

Response(response='\nFlash Attention是一种深度学习操作，它可以帮助计算机快速地理解文本中的关系。它可以比标准注意力更快地处理更长的序列，并且可以减少计算机需要访问的内存，从而提高计算机的性能。', source_nodes=[NodeWithScore(node=Node(text='in sequence length—than standard attention, thanks to the massively reduced amount of HBM access.\nWe analyze the IO complexity [ 1] ofFlashAttention , proving that it requires 𝑂¹𝑁2𝑑2𝑀\x001ºHBM\naccesses where 𝑑is the head dimension and 𝑀is the size of SRAM, as compared to Ω¹𝑁𝑑¸𝑁2ºof standard\nattention. For typical values of 𝑑and𝑀,FlashAttention requires many times fewer HBM accesses\ncompared to standard attention (up to 9 \x02fewer, as shown in Fig. 2). Moreover, we provide a lower bound,\nshowing that no exact attention algorithm can asymptotically improve on the number of HBM accesses over\nall SRAM sizes.\nWe also show that FlashAttention can serve as a useful primitive for realizing the potential of\napproximate attention algorithms by overcoming their issues with memory access overhead. As a proof of\nconcept, we implem